# Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json 
import torch
# import clip
from PIL import Image
# import sng_parser
from tqdm import tqdm 
import codecs
import numpy as np
import csv
import sys

from io import BytesIO
import base64

# Data

## Explore

In [16]:
csv.field_size_limit(sys.maxsize)

# path_data =  '/data/mshukor/data/ofa/pretrain_example/vision_language_examples.tsv'
# selected_cols='0,1,2,3,4,5,6,7'

# path_data =  '/data/mshukor/data/ofa/pretrain_example/detection_examples.tsv'
# selected_cols='0,1,2'

# path_data =  '/data/mshukor/data/ofa/pretrain_example/image_examples.tsv'
# selected_cols='0,1,2'

path_data =  '/data/mshukor/data/ofa/pretrain_example/text_examples.tsv'
selected_cols='0,1'

data_example = []

selected_col_ids = [int(col_id) for col_id in selected_cols.split(",")]

with open(path_data) as file:
    tsv_file = csv.reader(file, delimiter='\t')
    for line in tqdm(tsv_file):

        d = [line[i] for i in selected_col_ids]
#         print(d)
        data_example.append(d)
    


100it [00:00, 14325.30it/s]


In [17]:
line

['100',
 '...please depart this field clean unless you might be answering the question. do not ask questions you already know the answer to. thanks.retrieved from " " ad blocker interference detected! wikia is a single-to-usefulness web site that makes cash from promoting. we\'ve a experience for viewers using ad blockers wikia shouldn\'t be if youve made further modifications. take away the custom ad blocker (s) and the page leave timber as expected. categories : un-answered questionsadd class cancelsave per the reddit twine, flac files will be synced to an ios gadget via icloud impel, then accessed through thenew information utility , which will allow for local playback of the excessive-high quality audio files straight by the side of the device. if , it could stamp the first time that apple has offered help for the favored flac format an ios gadget.']

In [12]:
data_[6]

['7',
 'perhaps the clearest indication of who won and lost came quickly on the heels of the event itself: the democratic post-debate message was that joe biden scored a clear win; the republican message was that joe biden was too mean to paul ryan. the former is a boast of success; the latter is an excuse for failure. in the larger context, it’s hard to overstate how much democrats needed a shot in the arm like this. the surface-level goals of any vice presidential debate is for the candidates to demonstrate a capacity to step up in the event of a crisis, while defending their ticket’s agenda and knocking their rivals’ agenda. but for biden, the overarching benefit was about the basic morale of his party with less than four weeks to go until election day: he wanted to give democratic voters something to feel good about, and he did. who the hell am i! i’m a liberal that is extreme in some ways and not in others. i support president obama and make no apologies for it. i think he has don

In [ ]:
tasks = set()
datasets = set()

for d in data:
    tasks.add(d[-1])
    datasets.add(d[-2])

In [ ]:
int(data[10][0])

In [ ]:
# len(data[0][2:][0].split(' '))
# len(data[0][1])
text = data[10][1]
print(len(text.split(' ')))
print(len(text))
from nltk.tokenize import word_tokenize
len(word_tokenize(text))

In [ ]:
text

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
TreebankWordDetokenizer().detokenize(word_tokenize(text))

In [ ]:
key = 'refcoco_train'
index = -2
for d in data:
    if d[index] == key:
        print(d[2:])
#         break

In [ ]:
d[4].split(',')
str([287.0, 127.0, 340.0, 162.0])
'{:.2f},{:.2f},{:.2f},{:.2f}'.format(287.0, 127.0, 340.0, 162.0)

In [ ]:
print(len(data))
data[0]

In [ ]:
all_captions_path = '/data/mshukor/data/ofa/pretrain_example/negative_sample/all_captions.txt'
all_objects_path = '/data/mshukor/data/ofa/pretrain_example/negative_sample/object.txt'

all_object_list = [
    row.strip() for row in open(all_objects_path) if row.strip() != ''
]
all_caption_list = [
    row.strip() for row in open(all_captions_path) if row.strip() != ''
]


In [ ]:
len(all_object_list)

In [ ]:
len(all_caption_list)

In [ ]:
all_object_list[:10]

In [ ]:
all_caption_list[:10]

In [ ]:
json_path = '/data/mshukor/data/ofa/pretrain_example/negative_sample/type2ans.json'
type2ans = json.load(open(json_path,'r'))

In [ ]:
type2ans.keys()
# type2ans['what color is the']

### Our data

In [3]:
# path_data =  '/data/mshukor/data/ofa/pretrain_ours/text_mini.tsv'
# selected_cols='0,1'

path_data =  '/data/mshukor/data/ofa/pretrain_ours/detection_mini.tsv'
selected_cols='0,1,2'

# path_data =  '/data/mshukor/data/ofa/pretrain_ours/vision_language_mini.tsv'
# selected_cols='0,1,2,3,4,5,6,7'

# path_data =  '/data/mshukor/data/ofa/pretrain_ours/image_mini.tsv'
# selected_cols='0,1,2'

data = []

selected_col_ids = [int(col_id) for col_id in selected_cols.split(",")]

with open(path_data) as file:
    tsv_file = csv.reader(file, delimiter='\t')
    for line in tqdm(tsv_file):

        d = [line[i] for i in selected_col_ids]
#         print(d)
        data.append(d)

181767it [00:02, 70482.72it/s]


In [21]:
# new_data = []
# for d in data:
#     label_list = d[2].strip().split('&&')
#     new_label_list = []
#     for label in label_list:
#         lab = label.strip().split(',', 5)[:4] # x0, y0, x1, y1, cat_id, cat
        
#         if any(["&" in l for l in lab]):
#             lab = [remove_special(l) for l in lab]
            
#             print(lab)
#         lab_ = lab + label.strip().split(',', 5)[4:]
#         lab_ = ','.join(lab_)
#         new_label_list.append(lab_)
#     new_label_list = ['&&'.join(new_label_list)]
#     new_data.append(d[:2]+new_label_list)

In [24]:
for d in data:
    label_list = d[2].strip().split('&&')
    new_label_list = []
    for label in label_list:
        lab = label.strip().split(',', 5)[:4] # x0, y0, x1, y1, cat_id, cat
        
        if any(["&" in l for l in lab]):
            print(lab)
            # lab = [remove_special(l) for l in lab]

['&40.000', '155.000', '44.000', '164.000']


In [27]:
new_data[0]

['0',
 'coco/train2014/COCO_train2014_000000057870.jpg',
 '1.020,279.960,534.110,480.000,67,dining table&&90.670,271.490,262.510,480.000,62,chair&&233.290,270.450,403.610,473.810,62,chair&&367.820,264.270,506.970,480.000,62,chair&&476.760,261.030,596.490,462.740,62,chair&&263.030,174.370,417.670,299.400,64,potted plant&&539.330,290.160,640.000,469.210,62,chair&&10.790,260.030,125.120,384.070,62,chair&&560.800,413.950,639.090,479.200,67,dining table&&20.540,376.760,103.780,431.890,62,chair&&1.080,373.210,32.360,480.000,62,chair&&298.200,235.170,381.210,269.250,86,vase&&152.170,256.670,230.580,285.780,62,chair&&364.400,256.570,417.060,283.210,62,chair&&296.780,277.790,329.260,289.780,84,book&&292.800,289.310,314.210,300.650,84,book&&285.800,257.460,299.770,273.600,62,chair']

In [8]:
label_list

['1.020,279.960,534.110,480.000,67,dining table',
 '90.670,271.490,262.510,480.000,62,chair',
 '233.290,270.450,403.610,473.810,62,chair',
 '367.820,264.270,506.970,480.000,62,chair',
 '476.760,261.030,596.490,462.740,62,chair',
 '263.030,174.370,417.670,299.400,64,potted plant',
 '539.330,290.160,640.000,469.210,62,chair',
 '10.790,260.030,125.120,384.070,62,chair',
 '560.800,413.950,639.090,479.200,67,dining table',
 '20.540,376.760,103.780,431.890,62,chair',
 '1.080,373.210,32.360,480.000,62,chair',
 '298.200,235.170,381.210,269.250,86,vase',
 '152.170,256.670,230.580,285.780,62,chair',
 '364.400,256.570,417.060,283.210,62,chair',
 '296.780,277.790,329.260,289.780,84,book',
 '292.800,289.310,314.210,300.650,84,book',
 '285.800,257.460,299.770,273.600,62,chair']

In [12]:
def remove_special(input_string):
    final_string = ""
    for character in input_string:
        if  character == " ":
            final_string = final_string + character
        else:
            if(character.isalnum()):
                final_string = final_string + character
    return final_string

In [60]:
for d in tqdm(data):
    label = d[2]
    d[2] = remove_special(caption)


100%|█| 5593207/5593207 [00:35<00:0


In [4]:
data[0]

['0',
 'coco/train2014/COCO_train2014_000000057870.jpg',
 '1.020,279.960,534.110,480.000,67,dining table&&90.670,271.490,262.510,480.000,62,chair&&233.290,270.450,403.610,473.810,62,chair&&367.820,264.270,506.970,480.000,62,chair&&476.760,261.030,596.490,462.740,62,chair&&263.030,174.370,417.670,299.400,64,potted plant&&539.330,290.160,640.000,469.210,62,chair&&10.790,260.030,125.120,384.070,62,chair&&560.800,413.950,639.090,479.200,67,dining table&&20.540,376.760,103.780,431.890,62,chair&&1.080,373.210,32.360,480.000,62,chair&&298.200,235.170,381.210,269.250,86,vase&&152.170,256.670,230.580,285.780,62,chair&&364.400,256.570,417.060,283.210,62,chair&&296.780,277.790,329.260,289.780,84,book&&292.800,289.310,314.210,300.650,84,book&&285.800,257.460,299.770,273.600,62,chair']

In [78]:
for d in tqdm(data):
    d[2] = d[2].replace('\"', '')


100%|█| 181767/181767 [00:00<00:00,


In [49]:
data_  = []
with open(path_data) as file:
    for i in tqdm(range(6458670)):
        column_l = file.readline().rstrip("\n").split("\t")
        data_.append(column_l)
        if len(column_l) < 2:
            break

In [64]:

data_example  = []
fp = open('/data/mshukor/data/ofa/pretrain_ours/vision_language_mini_.tsv', "r")
data_example  = []
for line in tqdm(fp):
    data_example.append(line)

5593207it [00:03, 1463300.52it/s]


In [74]:
print(data_example[2796604])

2796604	cc3m/train/8/2d0d96e4ecb8e2e959a3bf10d59b9d05ac114aea.jpg	the residential development under construction in district				cc3m	caption



In [73]:
data_example[2796604]
fp.seek(2796604)
for l in fp:
    print(l)
    break

/val2014/COCO_val2014_000000329789.jpg	A young man is eating a slice of pizza in his room				coco_karp	caption



In [46]:
len(data_[10].rstrip("\n").split("\t")[1])# len(line)
# len(data_[10].rstrip("\n").split("\t")[1].encode('utf-8'))
print(len(data_example[10]), len(data_example[10].encode('utf-8')))
print(len(data_[10]), len(data_[10].encode('utf-8')), len(data_[10].encode('utf-8').decode("utf-8")))



2317 2317
2510 2514 2510


In [ ]:
print(data_[10].encode('utf-8'))
print(data_[10])

print(data_example[10].encode('utf-8'))
print(data_example[10])

In [4]:
output_path = '/data/mshukor/data/ofa/pretrain_ours/text_mini.tsv'

fp = open(output_path, "r")
data_  = []
for line in tqdm(fp):
    data_.append(line)
    

6458670it [01:45, 61129.36it/s] 


In [12]:
output_path = '/data/mshukor/data/ofa/pretrain_ours/text_mini.tsv'

start_id = 0 
num_max_characters = 2500

with open(output_path, 'w', newline='\n') as f_output:
    csv_output = csv.writer(f_output, delimiter='\t')

    for i, t in tqdm(enumerate(data)):
        text = t[1]
        item = [start_id, text]
        csv_output.writerow(item)
        start_id+=1

6458670it [04:08, 25941.37it/s]


In [28]:
output_path = '/data/mshukor/data/ofa/pretrain_ours/detection_mini.tsv'

with open(output_path, 'w', newline='\n') as f_output:
    csv_output = csv.writer(f_output, delimiter='\t')

    for t in tqdm(data):
        csv_output.writerow(t)

100%|█████████████████████████████████████████████████████████| 181767/181767 [00:03<00:00, 51934.09it/s]


In [63]:
t

['5593206',
 'train2014/COCO_train2014_000000524286.jpg',
 '',
 'Is that a laptop?',
 '1.0|!+yes',
 '',
 'vqa_train',
 'qa']

## Create data tsv

In [ ]:
def convert_img_to_str(file_name):
    img = Image.open(file_name) # path to file
    img_buffer = BytesIO()
    img.save(img_buffer, format=img.format)
    byte_data = img_buffer.getvalue()
    base64_str = base64.b64encode(byte_data) # bytes
    base64_str = base64_str.decode("utf-8") # str
    return base64_str

### Create VL tsv

#### Caption

In [ ]:
original_data_path = '/data/mshukor/data/our_albef_data/json_pretrain/sbu.json'
original_data = json.load(open(original_data_path,'r'))

In [ ]:
from preprocess.utils import get_tsv_data_from_jsons
    
datasets = [
            '/data/mshukor/data/our_albef_data/json_pretrain/coco_karp.json',
            '/data/mshukor/data/our_albef_data/json_pretrain/vg_albef.json',
            '/data/mshukor/data/our_albef_data/json_pretrain/sbu.json',
            '/data/mshukor/data/our_albef_data/json_pretrain/cc3m.json', 
    
            ['/data/mshukor/data/refcoco/refcoco+/refs(unc).p', '/data/mshukor/data/refcoco/refcoco+/instances.json'],
            
            '/data/mshukor/data/our_albef_data/data/vqa_train.json',
]

start_id = 0
task_types = ['caption',
             'caption',
             'caption',
             'caption',
             'visual_grounding',
             'qa',]

tsvs = get_tsv_data_from_jsons(datasets, start_id, task_types, convert_images=False)


In [ ]:
len(tsvs)
# tsvs[-10000]
tsvs[-1000000]

In [ ]:
import csv
from io import StringIO

output_path = '/data/mshukor/data/ofa/pretrain_ours/vision_language_mini.tsv'

with open(output_path, 'w', newline='') as f_output:
    csv_output = csv.writer(f_output, delimiter='\t')

    for t in tqdm(tsvs):
        csv_output.writerow(t)

In [ ]:
csv.field_size_limit(sys.maxsize)


out_data = []
selected_cols='0,1,2,3,4,5,6,7'

selected_col_ids = [int(col_id) for col_id in selected_cols.split(",")]

with open(output_path) as file:
    tsv_file = csv.reader(file, delimiter='\t')
    for line in tqdm(tsv_file):
        d = [line[i] for i in selected_col_ids]
        out_data.append(d)
    

In [ ]:
out_data[-1]

#### VQA

In [ ]:
original_data_path = '/data/mshukor/data/our_albef_data/data/vqa_train.json'
original_data = json.load(open(original_data_path,'r'))


In [ ]:
original_data[100]

In [ ]:
# 1.0|!+horizontal&&0.3|!+south&&0.3|!+straight&&0.3|!+vertical

from preprocess.utils import get_tsv_vqa_data_from_json


start_id = 0
dataset_name = 'vqav2'
task_type = 'qa'

image_root = '/data/mshukor/data/coco'
tmp = get_tsv_vqa_data_from_json(original_data, start_id, dataset_name, task_type, image_root=image_root, convert_images=False)

In [ ]:
tmp[10]

#### Visual Grounding 

In [ ]:
original_data_path = '/data/mshukor/data/our_albef_data/data/refcoco+_train.json'
original_data = json.load(open(original_data_path,'r'))

original_data_path = '/data/mshukor/data/our_albef_data/data/refcoco+/dets.json'
det_file = json.load(open(original_data_path,'r'))

original_data_path = '/data/mshukor/data/our_albef_data/data/refcoco+/cocos.json'
coco_file = json.load(open(original_data_path,'r'))

In [ ]:
list(det_file.keys())[:10]

In [ ]:
original_data_path = '/data/mshukor/data/refcoco/refcoco+/instances.json'
original_data = json.load(open(original_data_path,'r'))

In [ ]:
import pickle

ref_path = '/data/mshukor/data/refcoco/refcoco+/refs(unc).p'
refs = pickle.load(open(ref_path, 'rb'))

In [ ]:
for i, ref in tqdm(enumerate(refs)):
    
    

In [ ]:
len(refs)

In [ ]:
refs[500]

In [ ]:
id_to_annot = {}
for annot in original_data['annotations']:
    id_to_annot[annot['id']] = annot
    
    

In [ ]:
id_to_images = {}
for annot in tqdm(original_data['images']):
    id_to_images[annot['id']] = annot

In [ ]:
id_to_images[576457]

In [ ]:
list(id_to_annot.keys())[:10]
id_to_annot[1640859]['bbox']
for r in tqdm(id_to_annot.values()):
    if r['bbox'][0] > 0:
        print(r['bbox'])

In [ ]:
list(original_data.keys())[:10]

In [ ]:
ref_path = '/data/mshukor/data/refcoco/refcoco+/refs(unc).p'
instances_path = '/data/mshukor/data/refcoco/refcoco+/instances.json'
start_id = 0
dataset_name='refcoco_train'
task_type='visual_grounding'
convert_images=False
split='train'

tmp = get_tsv_from_refcoco(ref_path, instances_path, start_id, dataset_name=dataset_name, task_type=task_type, convert_images=convert_images, split=split)

In [ ]:
tmp[-1]

In [ ]:
Image.open('/data/mshukor/data/coco/train2014/COCO_train2014_000000000072.jpg').convert('RGB')

In [ ]:
original_data['images'][:10]

In [ ]:
# ['third book starting from left', '', '29.1,11.72,66.81,343.41', '', 'refcoco_train', 'visual_grounding']

original_data['categories']

### Imagenet

In [ ]:
# image-id and image base64 string .txt file 
# id, image, code in tsv final 

from preprocesss.utils import create_imagenet_txt_files


path_data = '/data/mshukor/data/imagenet/val'
output_path = '/data/mshukor/data/ofa/pretrain_ours/imagenet_val.txt'


create_imagenet_txt_files(path_data, output_path)

In [ ]:
start_id
len(data)
data[0]

In [ ]:

code_path = '/data/mshukor/data/ofa/pretrain_ours/imagenet_train_codes.tsv'
output_path = '/data/mshukor/data/ofa/pretrain_ours/image_mini.tsv'

def save_image_only_tsv_from_code_files(code_path, output_path, start_id=0):
    selected_col_ids = [0,1]
    out_data = []
    with open(code_path) as file:
        tsv_file = csv.reader(file, delimiter='\t')
        for line in tqdm(tsv_file):
            d = [line[i] for i in selected_col_ids]
            d = [start_id]+d
            out_data.append(d)


    with open(output_path, 'w', newline='') as f_output:
        csv_output = csv.writer(f_output, delimiter='\t')

        for t in tqdm(out_data):
            csv_output.writerow(t)

save_image_only_tsv_from_code_files(code_path, output_path, start_id=0)

In [ ]:
selected_col_ids = [0,1,2]
out_data = []
with open(output_path) as file:
    tsv_file = csv.reader(file, delimiter='\t')
    for line in tqdm(tsv_file):
        d = [line[i] for i in selected_col_ids]
        out_data.append(d)
        break

In [ ]:
len(out_data[0][2].split(' '))

#### Fix image paths

In [33]:

path_data =  '/data/mshukor/data/ofa/pretrain_ours/image_mini.tsv'
selected_cols='0,1,2'

data = []

selected_col_ids = [int(col_id) for col_id in selected_cols.split(",")]

with open(path_data) as file:
    tsv_file = csv.reader(file, delimiter='\t')
    for line in tqdm(tsv_file):

        d = [line[i] for i in selected_col_ids]
#         print(d)
        data.append(d)

1281167it [00:16, 79250.80it/s]


In [44]:
# from imge-id img-path to 
def replace_image_id_by_path(input_tsv, output_tsv, mapping_file):
    selected_cols='0,1,2'
    data = []
    selected_col_ids = [int(col_id) for col_id in selected_cols.split(",")]
    with open(input_tsv) as file:
        tsv_file = csv.reader(file, delimiter='\t')
        for line in tqdm(tsv_file):
            d = [line[i] for i in selected_col_ids]
            data.append(d)
            
    im_id_to_path = {}
    with open(mapping_file) as file:
        tsv_file = csv.reader(file, delimiter='\t')
        for line in tqdm(tsv_file):
            d = [line[i] for i in [0, 1]]
            im_id_to_path[d[0]] = d[1]
            
    for d in tqdm(data):
        im_id = d[1].split('/')[-1].split('.')[0]
        im_path = im_id_to_path[im_id]
        d[1] = im_path
        
    with open(output_tsv, 'w', newline='') as f_output:
        csv_output = csv.writer(f_output, delimiter='\t')

        for t in tqdm(data):
            csv_output.writerow(t)
        
    return data

input_tsv = '/data/mshukor/data/ofa/pretrain_ours/image_mini.tsv'
output_tsv = '/data/mshukor/data/ofa/pretrain_ours/image_mini.tsv'
mapping_file = '/data/mshukor/data/ofa/pretrain_ours/imagenet_train.txt'

tmp = replace_image_id_by_path(input_tsv, output_tsv, mapping_file)

1281167it [00:16, 76760.12it/s]
1281167it [00:01, 671149.72it/s]
100%|█████| 1281167/1281167 [00:01<00:00, 947543.73it/s]


In [45]:
tmp[0]

['0',
 'RawImages/train/n03146219/n03146219_8050.JPEG',
 '7442 662 7977 1652 6320 650 4376 992 1596 7734 1925 5335 3935 5604 5697 4504 5114 4050 144 215 144 6691 5321 7769 4755 3346 4691 3469 4175 1351 6907 9 6948 7749 7166 215 1026 931 970 4168 2675 6874 6248 2306 6138 8052 2970 6302 5550 2491 6931 969 6574 8014 6588 6639 389 1882 688 4691 4266 675 6248 3938 2387 4365 5999 261 2966 3499 651 5290 970 3526 5583 516 167 2103 1513 198 6657 7442 1118 7207 7307 1792 2078 388 4285 3417 5450 6959 6999 1306 1649 4556 2533 1103 6869 7681 8051 1916 7160 7743 2704 8063 2726 4860 2383 1635 8061 3497 7327 5915 7836 5697 1719 2136 96 970 7184 5167 2250 404 7007 7565 2742 33 7076 5250 7790 1838 1298 2847 3250 1204 1934 5550 4360 5688 1791 3465 634 4663 2991 5352 4066 4157 946 1596 3504 5855 5629 5411 7695 3627 3942 5631 2736 2883 5059 1423 2009 2643 1873 4960 1661 545 1396 3450 3145 211 6869 2226 6780 2724 4606 3702 3667 891 6236 6419 3531 7032 5277 3381 3031 7878 725 1652 1813 5037 949 3087 405 7884

In [36]:
# imgage_dir = 'imagenet/RawImages/train/'
# for d in tqdm(data):
#     im_id = d[1]
#     im_dir = im_id.split('_')[0]
#     im_path = os.path.join(im_dir, im_id+'.JPEG')
#     d[1] = os.path.join(imgage_dir, im_path)

100%|█████| 1281167/1281167 [00:03<00:00, 336250.44it/s]


In [39]:
data[0]

['0',
 'imagenet/RawImages/train/n03146219/n03146219_8050.JPEG',
 '7442 662 7977 1652 6320 650 4376 992 1596 7734 1925 5335 3935 5604 5697 4504 5114 4050 144 215 144 6691 5321 7769 4755 3346 4691 3469 4175 1351 6907 9 6948 7749 7166 215 1026 931 970 4168 2675 6874 6248 2306 6138 8052 2970 6302 5550 2491 6931 969 6574 8014 6588 6639 389 1882 688 4691 4266 675 6248 3938 2387 4365 5999 261 2966 3499 651 5290 970 3526 5583 516 167 2103 1513 198 6657 7442 1118 7207 7307 1792 2078 388 4285 3417 5450 6959 6999 1306 1649 4556 2533 1103 6869 7681 8051 1916 7160 7743 2704 8063 2726 4860 2383 1635 8061 3497 7327 5915 7836 5697 1719 2136 96 970 7184 5167 2250 404 7007 7565 2742 33 7076 5250 7790 1838 1298 2847 3250 1204 1934 5550 4360 5688 1791 3465 634 4663 2991 5352 4066 4157 946 1596 3504 5855 5629 5411 7695 3627 3942 5631 2736 2883 5059 1423 2009 2643 1873 4960 1661 545 1396 3450 3145 211 6869 2226 6780 2724 4606 3702 3667 891 6236 6419 3531 7032 5277 3381 3031 7878 725 1652 1813 5037 949 3087

In [40]:
output_path = '/data/mshukor/data/ofa/pretrain_ours/image_mini.tsv'
with open(output_path, 'w', newline='') as f_output:
    csv_output = csv.writer(f_output, delimiter='\t')

    for t in tqdm(data):
        csv_output.writerow(t)


100%|██████| 1281167/1281167 [00:27<00:00, 46704.02it/s]


### Object detection

#### COCO

In [ ]:
# '505.856,189.994,799.744,450.016,/m/07j7r,tree&&753.664,384.00600000000003,827.392,446.572,/m/0c9ph5,flower'

path_json = '/data/mshukor/data/coco/annotations/instances_train2014.json'

data = json.load(open(path_json,'r'))

In [ ]:
def get_tsv_from_coco_detection(instances_path, start_id, convert_images=True, split='train'):

    instances = json.load(open(instances_path,'r'))
    imgid_to_annot = {}
    for annot in tqdm(instances['annotations']):
        if annot['image_id'] not in imgid_to_annot:
            imgid_to_annot[annot['image_id']] = [annot]
        else:
            imgid_to_annot[annot['image_id']].append(annot)

    id_to_category = {}
    for annot in tqdm(instances['categories']):
        id_to_category[annot['id']] = annot['name']

    tsv_data = []
    missied = []
    for ref in tqdm(instances['images']):
        ref_split = split
        image_id = ref['id']
        file_name = ref['file_name']

        if ref_split == 'train':
            file_name = os.path.join('coco/train2014', file_name)

        if convert_images:
            img_path = os.path.join('/data/mshukor/data/', file_name)
            img = convert_img_to_str(img_path)
        else:
            img_path = file_name.replace('/data/mshukor/data/', '')
            img = img_path

        # ann_id = ref['id']
        # annot = id_to_annot[ann_id]
        if image_id not in imgid_to_annot:
            missied.append(image_id)
            continue
        annots = imgid_to_annot[image_id]
        detections = []
        areas = []
        for annot in annots:
            bbox = annot['bbox'] # x,y,w,h bottom left
            area = bbox[2]*bbox[3]
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]   # top left, bottom right 
            # box = '{:.3f},{:.3f},{:.3f},{:.3f}'.format(x1, y1, x2, y2)

            object_id = annot['category_id']
            category = id_to_category[object_id]

            tmp = '{:.3f},{:.3f},{:.3f},{:.3f},{},{}'.format(x1, y1, x2, y2, object_id, category)
            areas.append(area)
            detections.append(tmp)

        sorted_indices = sorted(range(len(areas)), key=lambda k: areas[k], reverse=True)
        detections = [detections[k] for k in sorted_indices]
        detections = '&&'.join(detections)
        t = [start_id, img, detections]

        tsv_data.append(t)
        start_id+=1

    return tsv_data

instances_path = '/data/mshukor/data/coco/annotations/instances_train2014.json'
start_id = 0
tmp = get_tsv_from_coco_detection(instances_path, start_id, convert_images=False, split='train')

In [ ]:
list(imgid_to_annot.keys())[:10]
len(missied)

#### VG

In [ ]:
def get_tsv_from_vg_detection(instances_path, path_images, start_id, convert_images=True, split='train'):
    
    instances = json.load(open(instances_path,'r'))
    
    id_to_objects = {}
    for d in instances:
        id_to_objects[d['id']] = d


    
    id_to_image_path = {}
    for root, dirs, files, in os.walk(path_images):
        for d in dirs:
            dir_path = os.path.join(root, d)
            for _, _, dir_files in os.walk(dir_path):
                for f in dir_files:
                    file_path = os.path.join(dir_path, f)
                    file_path = '/'.join(file_path.split('/')[-4:])
                    image_id = f.split('.')[0]
                    id_to_image_path[image_id] = file_path

                    


    tsv_data = []
    missied = []
    negs = []
    for ref in tqdm(id_to_image_path.keys()):
        ref_split = split
        
        image_id = ref
        
        file_name = id_to_image_path[image_id]
        if convert_images:
            img_path = os.path.join('/data/mshukor/data/', file_name)
            img = convert_img_to_str(img_path)
        else:
            img_path = file_name.replace('/data/mshukor/data/', '')
            img = img_path

            
        if int(image_id) in id_to_objects:
            objects = id_to_objects[int(image_id)]['objects']
        else:
            missied.append(image_id)
            continue
        
        if len(objects) == 0:
            missied.append(image_id)
            continue
            
        
        areas = []
        detections = []
        for annot in objects:
            x,y,w,h = annot['x'], annot['y'], annot['w'], annot['h'] # x,y,w,h bottom left
            
            area = w*h
            
            x1, y1, x2, y2 = x, y, x + w, y + h  # top left, bottom right 
            
            if x1 < 0 or x2 < 0:
                negs.append(annot)
            x1 = max(0, x1)
            x2 = max(0, x2)
            
            
            category = ','.join(annot['names']).replace('\x00','')
            object_id = annot['id']
            
            
            tmp = '{:.3f},{:.3f},{:.3f},{:.3f},{},{}'.format(x1, y1, x2, y2, object_id, category)
            detections.append(tmp)
            areas.append(area)

        sorted_indices = sorted(range(len(areas)), key=lambda k: areas[k], reverse=True)
        detections = [detections[k] for k in sorted_indices]
        
        detections = '&&'.join(detections)
        t = [start_id, img, detections]

        tsv_data.append(t)
        start_id+=1
    print('missed images:', len(missied), 'negs', len(negs))
    return tsv_data


instances_path = '/data/mshukor/data/visual_genome/annotations/objects.json'
path_images = '/data/mshukor/data/visual_genome/images'
start_id = 0

tmp = get_tsv_from_vg_detection(instances_path, path_images, start_id, convert_images=False, split='train')

In [ ]:
image_root = '/data/mshukor/data/'

Image.open(image_root+id_to_image_path['1087']).convert('RGB')

#### OpenImagesV5

In [ ]:
# data_path = '/data/mshukor/data/OpenImagesV5/train-annotations-bbox.csv'
# data_path = '/data/mshukor/data/OpenImagesV5/train-images-boxable.csv'
# data_path = '/data/mshukor/data/OpenImagesV5/train-images-boxable-with-rotation.csv'
data_path = '/data/mshukor/data/OpenImagesV5/class-descriptions-boxable.csv'




selected_col_ids = [0,1,2]
out_data = []
with open(data_path) as file:
    tsv_file = csv.reader(file, delimiter='\t')
    for i, line in tqdm(enumerate(tsv_file)):
        # d = [line[i] for i in selected_col_ids]
        out_data.append(line)
#         print(line)
#         if i > 2:
#             break
        

In [ ]:
def get_tsv_from_openimages_detection(instances_path, path_images, start_id, convert_images=False, split='train')

    id_to_image_path = {}
    for root, dirs, files, in os.walk(path_images):
        for d in dirs:
            dir_path = os.path.join(root, d)
            for _, _, dir_files in os.walk(dir_path):
                for f in dir_files:
                    file_path = os.path.join(dir_path, f)
                    file_path = '/'.join(file_path.split('/')[-4:])
                    image_id = f.split('.')[0]
                    id_to_image_path[image_id] = file_path

    image_root = '/gpfsdswork/dataset'

    def imagepath_to_image_size(path):
        w, h = Image.open(path).size

    id_to_annot = {}
    with open(instances_path) as file:
        tsv_file = csv.reader(file, delimiter='\t')
        for i, line in tqdm(enumerate(tsv_file)):
            img_id = line[0].split(',')[0]
            if img_id in id_to_annot:
                id_to_annot[img_id].append(line)
            else:
                id_to_annot[img_id] = [line]

    classid_to_class = {}

    with open(class_path) as file:
        tsv_file = csv.reader(file, delimiter=',')
        for i, line in tqdm(enumerate(tsv_file)):
            classid_to_class[line[0]] = line[1]

    tsv_data = []
    for img_id in id_to_annot.keys():
        annots = id_to_annot[img_id]
        img_path = id_to_image_path[img_id]
        orig_img_path = os.path.join(image_root, img_path)
        w, h = imagepath_to_image_size(path)

        if convert_images:
            img = convert_img_to_str(orig_img_path)
        else:
            img = img_path

        areas = []
        detections = []
        for d in annots:
            d = d[0].split(',')

            x1, x2, y1, y2 = d[4:8]
            x1, x2, y1, y2 = x1*w, x2*w, y1*h, y2*h
            box_w, box_h = x2 - x1, y2 - y1
            area = box_w*box_h
            areas.append(area)

            object_id = d[2]
            category = classid_to_class[object_id]

            tmp = '{:.3f},{:.3f},{:.3f},{:.3f},{},{}'.format(x1, y1, x2, y2, object_id, category)
            detections.append(tmp)


        sorted_indices = sorted(range(len(areas)), key=lambda k: areas[k], reverse=True)
        detections = [detections[k] for k in sorted_indices]

        detections = '&&'.join(detections)
        t = [start_id, img, detections]

        tsv_data.append(t)
        start_id+=1
        
    return tsv_data

    

In [ ]:
e39871fd9fd74f55

### Text

#### En Wikipedia

In [ ]:
from datasets import load_dataset

In [ ]:
%env http_proxy='http://192.168.0.100:3128' 
%env https_proxy='http://192.168.0.100:3128'

%env HF_DATASETS_CACHE="/data/mshukor/data/.cache"
%env HF_DATASETS_OFFLINE=1

In [ ]:
tmp = load_dataset("wikipedia", "20220301.en", cache_dir="/data/mshukor/data/.cache")

In [ ]:
len(tmp['train'][0]['text'])
tmp['train'][0]['text'][:512]

In [ ]:
def remove_special(input_string):
    final_string = ""
    for character in input_string:
        if  character == " ":
            final_string = final_string + character
        else:
            if(character.isalnum()):
                final_string = final_string + character
    return final_string

def get_tsv_from_text_data(data_name="wikipedia", data_subname="20220301.en", 
                           output_path, cache_dir="/data/mshukor/data/.cache", start_id=0, num_max_characters=2500):
    from datasets import load_dataset
    tmp = load_dataset(data_name, data_subname, cache_dir=cache_dir)

    with open(output_path, 'w', newline='') as f_output:
        csv_output = csv.writer(f_output, delimiter='\t')

        for i, t in tqdm(enumerate(tmp['train'])):
            text = t['text'][:num_max_characters].replace('\t', ' ').replace("\n", ' ').replace('\"', '')
            text = remove_special(text)
            item = [start_id, text]
            csv_output.writerow(item)
            start_id+=1

    

In [ ]:
import csv
from io import StringIO

output_path = '/data/mshukor/data/ofa/pretrain_ours/text_mini.tsv'

start_id = 0 
num_max_characters = 2500

with open(output_path, 'w', newline='') as f_output:
    csv_output = csv.writer(f_output, delimiter='\t')

    for i, t in tqdm(enumerate(tmp['train'])):
        text = t['text'][:num_max_characters]
        item = [start_id, text]
        csv_output.writerow(item)
        start_id+=1

In [ ]:
out_data = []
selected_cols='0,1'

selected_col_ids = [int(col_id) for col_id in selected_cols.split(",")]

with open(output_path) as file:
    tsv_file = csv.reader(file, delimiter='\t')
    for line in tqdm(tsv_file):
        d = [line[i] for i in selected_col_ids]
        out_data.append(d)
    

In [ ]:
out_data

### Create from finetuned data

In [ ]:
read from tsv and write to tsv directly 
same for vqa v2
then create ofa_mini 4m, vqa and refcoco for pretraining 

# Convert weights

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from models import ofa_base_architecture, OFAModel
from transformers.tokenization_utils_base import BatchEncoding

### Explore ofa

In [4]:
import torch
import numpy as np
from fairseq import utils, tasks
from fairseq import checkpoint_utils
from utils.eval_utils import eval_step
from tasks.mm_tasks.caption import CaptionTask
from models.ofa import OFAModel
from PIL import Image

# Register refcoco task
tasks.register_task('caption', CaptionTask)

# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False

# Load pretrained ckpt & config
overrides={"eval_cider":False, "beam":5, "max_len_b":16, "no_repeat_ngram_size":3, "seed":7}
models, cfg, task = checkpoint_utils.load_model_ensemble_and_task(
        utils.split_paths('/data/mshukor/logs/ofa/checkpoints/caption/ofa_caption_stage_1/5_0.06_6000/checkpoint_best.pt'),
        arg_overrides=overrides
    )

# Move models to GPU
for model in models:
    model.eval()
    if use_fp16:
        model.half()
    if use_cuda and not cfg.distributed_training.pipeline_model_parallel:
        model.cuda()
    model.prepare_for_inference_(cfg)

# Initialize generator
generator = task.build_generator(models, cfg.generation)

2022-11-15 08:52:08 | INFO | tasks.ofa_task | source dictionary: 59457 types
2022-11-15 08:52:08 | INFO | tasks.ofa_task | target dictionary: 59457 types


In [5]:
model_ofa = models[0]
ofa_state = model_ofa.state_dict()

In [ ]:
def get_state_given_key(state, key, excluded_keys=None):
    new_state = {}
    for k, v in state.items():
        if key in k:
            if excluded_keys is not None:
                if not any([ek in k for ek in excluded_keys]):
                    new_state[k] = v
            else:
                new_state[k] = v
    return new_state

key = 'encoder.layers.0'
excluded_keys = ['embed', 'image']
ofa_tmp = get_state_given_key(ofa_state, key, excluded_keys=excluded_keys)

In [ ]:
# def get_ofa_args_large(args):
#     args['encoder_embed_path'] = getattr(args, "encoder_embed_path", None)
#     args['encoder_embed_dim'] = getattr(args, "encoder_embed_dim", 1024)
#     args['encoder_ffn_embed_dim'] = getattr(args, "encoder_ffn_embed_dim", 4 * 1024)
#     args['encoder_layers'] = getattr(args, "encoder_layers", 12)
#     args['encoder_attention_heads'] = getattr(args, "encoder_attention_heads", 16)
#     args['encoder_normalize_before'] = getattr(args, "encoder_normalize_before", True)
#     args['encoder_learned_pos'] = getattr(args, "encoder_learned_pos", True)
#     args['decoder_embed_path'] = getattr(args, "decoder_embed_path", None)
#     args['decoder_embed_dim'] = getattr(args, "decoder_embed_dim", args['encoder_embed_dim'])
#     args['decoder_ffn_embed_dim'] = getattr(
#         args, "decoder_ffn_embed_dim", args['encoder_ffn_embed_dim']
#     )
#     args['decoder_layers'] = getattr(args, "decoder_layers", 12)
#     args['decoder_attention_heads'] = getattr(args, "decoder_attention_heads", 16)
#     args['decoder_normalize_before'] = getattr(args, "decoder_normalize_before", True)
#     args['decoder_learned_pos'] = getattr(args, "decoder_learned_pos", True)
#     args['attention_dropout'] = getattr(args, "attention_dropout", 0.0)
#     args['relu_dropout'] = getattr(args, "relu_dropout", 0.0)
#     args['dropout'] = getattr(args, "dropout", 0.0)
#     args['max_target_positions'] = getattr(args, "max_target_positions", 1024)
#     args['max_source_positions'] = getattr(args, "max_source_positions", 1024)
#     args['adaptive_softmax_cutoff'] = getattr(args, "adaptive_softmax_cutoff", None)
#     args['adaptive_softmax_dropout'] = getattr(args, "adaptive_softmax_dropout", 0)
#     args['share_decoder_input_output_embed'] = getattr(
#         args, "share_decoder_input_output_embed", True
#     )
#     args['share_all_embeddings'] = getattr(args, "share_all_embeddings", True)

#     args['decoder_output_dim'] = getattr(
#         args, "decoder_output_dim", args['decoder_embed_dim']
#     )
#     args['decoder_input_dim'] = getattr(args, "decoder_input_dim", args['decoder_embed_dim'])

#     args['no_scale_embedding'] = getattr(args, "no_scale_embedding", True)
#     args['layernorm_embedding'] = getattr(args, "layernorm_embedding", True)

#     args['activation_fn'] = getattr(args, "activation_fn", "gelu")
#     args['pooler_activation_fn'] = getattr(args, "pooler_activation_fn", "tanh")
#     args['pooler_dropout'] = getattr(args, "pooler_dropout", 0.0)
#     args['pooler_classifier'] = getattr(args, "pooler_classifier", "mlp")

#     args['resnet_drop_path_rate'] = getattr(args, "resnet_drop_path_rate", 0.0)
#     args['encoder_drop_path_rate'] = getattr(args, "encoder_drop_path_rate", 0.0)
#     args['decoder_drop_path_rate'] = getattr(args, "decoder_drop_path_rate", 0.0)

#     args['resnet_type'] = getattr(args, "resnet_type", "resnet152")
#     args['token_bucket_size'] = getattr(args, "token_bucket_size", 256)
#     args['image_bucket_size'] = getattr(args, "image_bucket_size", 42)

#     args['freeze_encoder_embedding'] = getattr(args, "freeze_encoder_embedding", False)
#     args['freeze_decoder_embedding'] = getattr(args, "freeze_decoder_embedding", False)
#     args['add_type_embedding'] = getattr(args, "add_type_embedding", True)
#     args['attn_scale_factor'] = getattr(args, "attn_scale_factor", 2)

#     args['code_image_size'] = getattr(args, "code_image_size", 128)
#     args['patch_layernorm_embedding'] = getattr(args, "patch_layernorm_embedding", True)
#     args['code_layernorm_embedding'] = getattr(args, "code_layernorm_embedding", True)
#     args['entangle_position_embedding'] = getattr(args, "entangle_position_embedding", False)
#     args['disable_entangle'] = getattr(args, "disable_entangle", False)
#     args['sync_bn'] = getattr(args, "sync_bn", False)

#     args['scale_attn'] = getattr(args, "scale_attn", False)
#     args['scale_fc'] = getattr(args, "scale_fc", False)
#     args['scale_heads'] = getattr(args, "scale_heads", False)
#     args['scale_resids'] = getattr(args, "scale_resids", False)

#     args['orig_patch_image_size'] = getattr(args, "orig_patch_image_size", 256)

#     return args

In [ ]:
# args = {}
# args = get_ofa_args_large(args)
# args = BatchEncoding(args)
# ofa_base_architecture(args)
# data_dir = '/data/mshukor/data/ofa/pretrain_example'

# cfg.task.neg_sample_dir = data_dir+'/negative_sample'

### convert t5 weights

In [6]:
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-base")

In [ ]:
model_t5

In [7]:
t5_state = model_t5.state_dict()

In [56]:
import re
# line = re.sub(r"</?\[\d+>", "", line)

mapping_dict = {
    ## encoder
    'block': 'layers', 
    'layer.[0-9]+.SelfAttention': 'self_attn', 
    '.q.': '.q_proj.', 
    '.k.weight': '.k_proj.weight', 
    '.v.': '.v_proj.', 
    # '.o.weight': '.out_proj.weight',    
    'layer.0.layer_norm.': 'self_attn_layer_norm.', 
    'layer.[0-9]+.DenseReluDense.': '', 
    '.wi.': '.fc1.', 
    '.wo.': '.fc2.', 
    
    
    # decoder
    'layer.[0-9]+.EncDecAttention': 'encoder_attn', 
    # 'layer.1.layer_norm.': 'encoder_attn_layer_norm.', 
    
    
}

encoder_mapping = {
    'layer.1.layer_norm.': 'final_layer_norm.',   
}

decoder_mapping = {
    'layer.1.layer_norm.': 'encoder_attn_layer_norm.', 
    'layer.2.layer_norm.': 'final_layer_norm.', 
}


simple_replace_mapping = {
    
 '.o.weight': '.out_proj.weight',    
}
def modify_state(state, mapping_dict, encoder_mapping, decoder_mapping, simple_replace_mapping):
    # orig_keys = ['block', 'layer.[0-9]+.SelfAttention', '.q.', '.k.', '.v.', '.o.', '0.layer_norm.', '.DenseReluDense.wi.', '.DenseReluDense.wo.', '.1.layer_norm.']
    # new_keys = ['layers', 'layer.self_attn', '.q_proj.', '.k_proj.', '.v_proj.', '.out_proj.', '.self_attn_layer_norm.', '.fc1.', '.fc2.', '.final_layer_norm.']
    
    new_state = state.copy()
    old_keys = []
    for k, v in state.items():
        
        new_key = '%s' % k  
        for old, new in simple_replace_mapping.items():
            new_key = new_key.replace(old, new)
            
        for old, new in mapping_dict.items():
            new_key = re.sub(r"{}".format(old), new, new_key)
            
        if 'encoder' in new_key:
            for old, new in encoder_mapping.items():
                new_key = re.sub(r"{}".format(old), new, new_key)
            
        if 'decoder' in new_key:
            for old, new in decoder_mapping.items():
                new_key = re.sub(r"{}".format(old), new, new_key)
        
        new_state[new_key] = v
        old_keys.append(k)
        
        
    
    
    for k in old_keys:
        del new_state[k]
        
    final_state = {}
    final_state['model'] = new_state
    return final_state
    
new_state = modify_state(t5_state, mapping_dict, encoder_mapping, decoder_mapping, simple_replace_mapping)



In [ ]:
new_state['model'].keys()

In [ ]:
def compare_states(state1, state2):
    different = []
    for k1 in state1.keys():
        if k1 not in state2:
            different.append(k1)
    return different
            
tmp = compare_states(new_state, ofa_state)

In [35]:
output_path = '/data/mshukor/logs/ofa/pretrained_models/t5_base.pt'
torch.save(new_state, output_path)

In [51]:
output_path = '/data/mshukor/logs/ofa/pretrained_models/t5_base.pt'

tmp_state = torch.load(output_path)

In [ ]:

model_ofa.load_state_dict(tmp_state['model'],  strict=False)

In [ ]:
tmp_state['model'].keys()

In [18]:
tmp_state = torch.load('/data/mshukor/logs/ofa/pretrained_models/ofa_base.pt')


In [19]:
tmp_state.keys()

dict_keys(['args', 'cfg', 'model', 'criterion', 'optimizer_history', 'task_state', 'extra_state', 'last_optimizer_state'])

In [ ]:
model_t5.encoder.block[0]

In [ ]:
model_ofa.encoder.layers[0]

### convert BART weights

In [7]:
weights_path = '/data/mshukor/logs/ofa/pretrained_models/bart.base/model.pt'
bart_state = torch.load(weights_path, map_location=torch.device('cpu'))

In [13]:
model_ofa.load_state_dict(bart_state['model'],  strict=True)

<All keys matched successfully>

In [9]:
bart_state['model'].keys()

odict_keys(['encoder.version', 'encoder.embed_tokens.weight', 'encoder.embed_positions.weight', 'encoder.layers.0.self_attn.k_proj.weight', 'encoder.layers.0.self_attn.k_proj.bias', 'encoder.layers.0.self_attn.v_proj.weight', 'encoder.layers.0.self_attn.v_proj.bias', 'encoder.layers.0.self_attn.q_proj.weight', 'encoder.layers.0.self_attn.q_proj.bias', 'encoder.layers.0.self_attn.out_proj.weight', 'encoder.layers.0.self_attn.out_proj.bias', 'encoder.layers.0.self_attn_layer_norm.weight', 'encoder.layers.0.self_attn_layer_norm.bias', 'encoder.layers.0.fc1.weight', 'encoder.layers.0.fc1.bias', 'encoder.layers.0.fc2.weight', 'encoder.layers.0.fc2.bias', 'encoder.layers.0.final_layer_norm.weight', 'encoder.layers.0.final_layer_norm.bias', 'encoder.layers.1.self_attn.k_proj.weight', 'encoder.layers.1.self_attn.k_proj.bias', 'encoder.layers.1.self_attn.v_proj.weight', 'encoder.layers.1.self_attn.v_proj.bias', 'encoder.layers.1.self_attn.q_proj.weight', 'encoder.layers.1.self_attn.q_proj.bias'

In [12]:
ofa_state.keys()
ofa_state['encoder.embed_tokens.weight']

tensor([[ 0.0125,  0.0014, -0.0096,  ...,  0.0022,  0.1057,  0.0103],
        [-0.0114, -0.0169, -0.0184,  ..., -0.0131, -0.0043, -0.0053],
        [ 0.0842, -0.0389,  0.0096,  ...,  0.0583,  0.0082,  0.0357],
        ...,
        [-0.0032, -0.0313, -0.1026,  ...,  0.0138,  0.0056, -0.0023],
        [ 0.0104, -0.0045,  0.0263,  ...,  0.0158,  0.0324, -0.0111],
        [-0.0038, -0.0532, -0.0147,  ...,  0.0067,  0.0256,  0.0009]])